In [1]:
def check_digit(x):
    try:
        odd = map(int, ','.join(x[-1::-2]).split(','))
        even = map(int, ','.join(x[-2::-2]).split(','))
        sum_odd3 = sum(odd) * 3
        total = sum_odd3 + sum(even)
        rem = total % 10
        if rem == 0:
            return rem
        return 10 - rem
    except:
        return -9999

In [2]:
def category_counts(data):
    """
    Count total number of unique DepartmentDescription made on each trip.
    """
    counts = []
    for array in np.asarray(data.loc[:, "1-HR PHOTO":"WIRELESS"]):
        count = 0
        for item in array:
            if item > 0:
                count += 1
        counts.append(count)
    cat_counts = pd.DataFrame(counts)
#     cat_counts = cat_counts.rename(columns={0:"CategoryCounts"})
    cat_counts = cat_counts.set_index(data.index)

    
    data.insert(6, "CategoryCounts", cat_counts)
    
    return data

In [3]:
def company(x):
    try:
        p = x[:6]
        if p == "000000":
            return x[-5]
        return p
    except:
        return -9999
    
def float_to_str(obj):
    while obj != "nan":
        obj = str(obj).split(".")[0]
        return obj

In [4]:
from collections import Counter
def mode(x):
    counts = Counter(x)
    max_count = max(counts.values())
    ls = [x_i for x_i, count in counts.items() if count == max_count]
    return ls[0]

In [88]:
train = pd.read_csv('../../data/train.csv')
test = pd.read_csv('../../data/test.csv')

In [92]:
df_num = train.select_dtypes(include = ['float64', 'int64'])
df_num.head()

,TripType,VisitNumber,Upc,ScanCount,FinelineNumber
0,999,5,6.811315e+10,-1,1000.0
1,30,7,6.053882e+10,1,8931.0
2,30,7,7.410811e+09,1,4504.0
3,26,8,2.238404e+09,2,3565.0
4,26,8,2.006614e+09,2,1017.0


In [6]:
train["Upc"] = train.Upc.apply(float_to_str)
test["Upc"] = test.Upc.apply(float_to_str)

train["company"] = train.Upc.apply(company) 
test["company"] = test.Upc.apply(company)

In [7]:
train.loc[train["ScanCount"] < 0, "Return"] = 1
train.loc[train["Return"] != 1, "Return"] = 0

test.loc[test["ScanCount"] < 0, "Return"] = 1
test.loc[test["Return"] != 1, "Return"] = 0

In [8]:
wd = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, 
      "Friday": 4, "Saturday": 5, "Sunday": 6}

train["Weekday"] = train["Weekday"].apply(lambda x: wd[x])
test["Weekday"] = test["Weekday"].apply(lambda x: wd[x])

In [9]:
train["Pos_Sum"] = train["ScanCount"]
test["Pos_Sum"] = test["ScanCount"]

train.loc[train["Pos_Sum"] < 0, "Pos_Sum"] = 0
test.loc[test["Pos_Sum"] < 0, "Pos_Sum"] = 0

In [10]:
train["Neg_Sum"] = train["ScanCount"]
test["Neg_Sum"] = test["ScanCount"]

train.loc[train["Neg_Sum"] > 0, "Neg_Sum"] = 0
test.loc[test["Neg_Sum"] > 0, "Neg_Sum"] = 0

In [11]:
train["check"] = train.Upc.apply(check_digit)
test["check"] = test.Upc.apply(check_digit)

In [12]:
# new feature

train_fineline = train[["VisitNumber", "FinelineNumber"]]
test_fineline = test[["VisitNumber", "FinelineNumber"]]

train_fineline = train_fineline.groupby("VisitNumber", as_index=False).agg(mode)
train_fineline.rename(columns={"FinelineNumber": "MF_FinelineNumber"}, inplace=True)

test_fineline = test_fineline.groupby("VisitNumber", as_index=False).agg(mode)
test_fineline.rename(columns={"FinelineNumber": "MF_FinelineNumber"}, inplace=True)

In [13]:
train_upc_fine = train[["VisitNumber", "Upc", "FinelineNumber"]]
test_upc_fine = test[["VisitNumber", "Upc", "FinelineNumber"]]

train_upc_fine_group = train_upc_fine.groupby("VisitNumber", as_index=False).count()
train_upc_fine_group.rename(columns={"Upc": "N_Upc", 
                                     "FinelineNumber": "N_FinelineNumber"}, inplace=True)
test_upc_fine_group = test_upc_fine.groupby("VisitNumber", as_index=False).count()
test_upc_fine_group.rename(columns={"Upc": "N_Upc", 
                                    "FinelineNumber": "N_FinelineNumber"}, inplace=True)

In [14]:
train = train.drop(["Upc", "FinelineNumber"], axis=1)
test = test.drop(["Upc", "FinelineNumber"], axis=1)

In [15]:
train_dd = pd.get_dummies(train["DepartmentDescription"])
test_dd = pd.get_dummies(test["DepartmentDescription"])

train_dd = pd.concat([train[["VisitNumber"]], train_dd], axis=1)
test_dd = pd.concat([test[["VisitNumber"]], test_dd], axis=1)

train_dd = train_dd.groupby("VisitNumber", as_index=False).sum()
test_dd = test_dd.groupby("VisitNumber", as_index=False).sum()

In [16]:
train_dd.shape, test_dd.shape

((95674, 69), (95674, 68))

In [17]:
train_company = train[["VisitNumber", "company"]]
test_company = test[["VisitNumber", "company"]]

train_company = train_company.groupby("VisitNumber", as_index=False).agg(mode)
test_company = test_company.groupby("VisitNumber", as_index=False).agg(mode)

In [18]:
train_company.shape, test_company.shape

((95674, 2), (95674, 2))

In [19]:
train_check = train[["VisitNumber", "check"]]
test_check = test[["VisitNumber", "check"]]

train_check = train_check.groupby("VisitNumber", as_index=False).agg(mode)
test_check = test_check.groupby("VisitNumber", as_index=False).agg(mode)

In [20]:
train_by_sum = train[["VisitNumber", "ScanCount", "Pos_Sum", "Neg_Sum"]]
test_by_sum = test[["VisitNumber", "ScanCount", "Pos_Sum", "Neg_Sum"]]

train_by_max = train[["TripType", "VisitNumber", "Weekday", "Return"]]
test_by_max = test[["VisitNumber", "Weekday", "Return"]]

In [21]:
train_by_sum = train_by_sum.groupby("VisitNumber", as_index=False).sum()
test_by_sum = test_by_sum.groupby("VisitNumber", as_index=False).sum()

train_by_max = train_by_max.groupby("VisitNumber", as_index=False).max()
test_by_max = test_by_max.groupby("VisitNumber", as_index=False).max()

In [22]:
train = train_by_sum.merge(train_by_max, on=["VisitNumber"])
train = train.merge(train_dd, on=["VisitNumber"])
train = train.merge(train_company, on=["VisitNumber"])
train = train.merge(train_check, on=["VisitNumber"])
train = train.merge(train_fineline, on=["VisitNumber"])
train = train.merge(train_upc_fine_group, on=["VisitNumber"])

test = test_by_sum.merge(test_by_max, on=["VisitNumber"])
test = test.merge(test_dd, on=["VisitNumber"])
test = test.merge(test_company, on=["VisitNumber"])
test = test.merge(test_check, on=["VisitNumber"])
test = test.merge(test_fineline, on=["VisitNumber"])
test = test.merge(test_upc_fine_group, on=["VisitNumber"])

In [23]:
y = train["TripType"]
train = train.drop(["TripType", "HEALTH AND BEAUTY AIDS"], axis=1)

In [24]:
%%time
train = category_counts(train)
test = category_counts(test)

CPU times: user 25 s, sys: 151 ms, total: 25.1 s
Wall time: 25.4 s


In [25]:
train["Upc_Depts"] = train["N_Upc"] / train["N_FinelineNumber"]
test["Upc_Depts"] = test["N_Upc"] / test["N_FinelineNumber"]

In [26]:
train = pd.get_dummies(train, columns=["Weekday", "Return"])
test = pd.get_dummies(test, columns=["Weekday", "Return"])

In [27]:
train.shape, test.shape

((95674, 87), (95674, 87))

In [36]:
train.head()

,VisitNumber,ScanCount,Pos_Sum,Neg_Sum,CategoryCounts,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,...,Upc_Depts,Weekday_0,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Return_0.0,Return_1.0
0,5,-1,0,-1,1,0,0,0,0,0,...,1.000000,0,0,0,0,1,0,0,0,1
1,7,2,2,0,2,0,0,0,0,0,...,1.000000,0,0,0,0,1,0,0,1,0
2,8,28,30,-2,6,0,0,0,0,0,...,1.045455,0,0,0,0,1,0,0,0,1
3,9,3,3,0,2,0,0,0,0,0,...,1.000000,0,0,0,0,1,0,0,1,0
4,10,3,3,0,2,0,0,0,0,0,...,1.000000,0,0,0,0,1,0,0,1,0


In [37]:
test.head()

,VisitNumber,ScanCount,Pos_Sum,Neg_Sum,CategoryCounts,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,...,Upc_Depts,Weekday_0,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Return_0.0,Return_1.0
0,1,4,4,0,3,0,0,0,0,0,...,1.0,0,0,0,0,1,0,0,1,0
1,2,4,4,0,3,0,0,0,1,0,...,1.0,0,0,0,0,1,0,0,1,0
2,3,0,1,-1,1,0,0,0,0,0,...,1.0,0,0,0,0,1,0,0,0,1
3,4,1,1,0,1,0,0,0,0,0,...,1.0,0,0,0,0,1,0,0,1,0
4,6,0,1,-1,1,0,0,0,0,0,...,1.0,0,0,0,0,1,0,0,0,1


In [60]:
y

0        999
1         30
2         26
3          8
4          8
5         35
6         41
7         21
8          6
9         42
10         7
11         9
12         8
13        39
14        25
15         8
16         9
17         8
18         8
19         8
20         8
21         8
22         8
23        38
24         8
25        35
26        15
27         9
28         8
29         8
        ... 
95644     21
95645     44
95646      8
95647     44
95648     40
95649     26
95650      8
95651      8
95652     36
95653     40
95654     42
95655     36
95656     31
95657      9
95658     42
95659     30
95660      7
95661     38
95662     40
95663     38
95664     24
95665      9
95666     32
95667     38
95668      8
95669     25
95670     22
95671     39
95672     39
95673      8
Name: TripType, Length: 95674, dtype: int64

In [28]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [29]:
label_enc = LabelEncoder().fit(y)
y_labeled = label_enc.transform(y)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    train, y_labeled, random_state=0)

In [31]:
dtrain = xgb.DMatrix(X_train.values, label=y_train)
dtest = xgb.DMatrix(X_test.values, label=y_test)

In [50]:
num_boost_round = 300
params = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class':38, 
          'max_delta_step': 3, 
          'eta': 0.25,
          'max_features': 10, 
          'min_samples_leaf': 3,
          'min_samples_split': 2}

evals = [(dtrain, 'train'), (dtest, 'eval')]


bst = xgb.train(params=params,  
                dtrain=dtrain, 
                num_boost_round=num_boost_round, 
                evals=evals,
               early_stopping_rounds=10,)

[0]	train-mlogloss:2.93537	eval-mlogloss:2.9471
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:2.34326	eval-mlogloss:2.37095
[2]	train-mlogloss:1.91318	eval-mlogloss:1.95558
[3]	train-mlogloss:1.65661	eval-mlogloss:1.71024
[4]	train-mlogloss:1.48188	eval-mlogloss:1.54478
[5]	train-mlogloss:1.34987	eval-mlogloss:1.42094
[6]	train-mlogloss:1.24623	eval-mlogloss:1.32529
[7]	train-mlogloss:1.16356	eval-mlogloss:1.24927
[8]	train-mlogloss:1.09548	eval-mlogloss:1.18797
[9]	train-mlogloss:1.03786	eval-mlogloss:1.13655
[10]	train-mlogloss:0.9907	eval-mlogloss:1.09534
[11]	train-mlogloss:0.949775	eval-mlogloss:1.06015
[12]	train-mlogloss:0.915182	eval-mlogloss:1.03132
[13]	train-mlogloss:0.883135	eval-mlogloss:1.00522
[14]	train-mlogloss:0.855546	eval-mlogloss:0.983329
[15]	train-mlogloss:0.830766	eval-mlogloss:0.963848
[16]	train-mlogloss:0.809331	eval-mlogloss:0.947327
[1

In [56]:
num_boost_round = 300
params = {'objective': 'multi:softmax', 
#          'eval_metric': 'mlogloss',
          'num_class':38, 
          'max_delta_step': 3, 
          'eta': 0.25,
          'max_features': 10, 
          'min_samples_leaf': 3,
          'min_samples_split': 2}

evals = [(dtrain, 'train'), (dtest, 'eval')]


bst = xgb.train(params=params,  
                dtrain=dtrain, 
                num_boost_round=num_boost_round, 
                evals=evals,
               early_stopping_rounds=10,)

[0]	train-merror:0.458142	eval-merror:0.467913
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.349481	eval-merror:0.363853
[2]	train-merror:0.317386	eval-merror:0.341235
[3]	train-merror:0.300495	eval-merror:0.32656
[4]	train-merror:0.292635	eval-merror:0.321209
[5]	train-merror:0.28805	eval-merror:0.318032
[6]	train-merror:0.280886	eval-merror:0.312304
[7]	train-merror:0.274504	eval-merror:0.307413
[8]	train-merror:0.27134	eval-merror:0.303566
[9]	train-merror:0.265779	eval-merror:0.301183
[10]	train-merror:0.262044	eval-merror:0.299135
[11]	train-merror:0.257153	eval-merror:0.296877
[12]	train-merror:0.252526	eval-merror:0.294996
[13]	train-merror:0.24861	eval-merror:0.293616
[14]	train-merror:0.245335	eval-merror:0.292571
[15]	train-merror:0.2416	eval-merror:0.289937
[16]	train-merror:0.238938	eval-merror:0.288766
[17]	train-merror:0.236011	eval-merror:0.287261
[18]	

In [54]:
from sklearn.metrics import log_loss
my_test = xgb.DMatrix(X_test.values)
test_predictions = bst.predict(my_test)
log_loss(y_test, test_predictions).round(5)

0.78525

In [57]:
test_predictions2 = bst.predict(my_test).astype(int)

In [59]:
from sklearn.metrics import classification_report
print(classification_report(np.array(y_test), test_predictions2))

             precision    recall  f1-score   support

          0       0.94      0.99      0.97       931
          1       0.84      0.55      0.67        87
          2       0.80      0.84      0.82      1167
          3       0.78      0.80      0.79       309
          4       0.74      0.72      0.73      1494
          5       0.84      0.89      0.87      3017
          6       0.78      0.84      0.81      2307
          7       0.46      0.21      0.29        62
          8       0.00      0.00      0.00         2
          9       0.62      0.43      0.51       237
         10       0.41      0.40      0.40       131
         11       0.65      0.60      0.62        80
         12       0.65      0.58      0.62       173
         13       0.68      0.67      0.68       158
         14       0.53      0.56      0.54       207
         15       0.72      0.60      0.66        43
         16       0.62      0.67      0.65       652
         17       0.72      0.77      0.74   

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [81]:
yprob = test_predictions.reshape(y_test.shape[0], 38)
ylabel = np.argmax(yprob, axis=1)
accuracy = 100 *(1 - (sum(int(ylabel[i]) != y_test[i]
                         for i in range(len(y_test))) / float(len(y_test))))
print("TripType Accuracy = %.2f%%" % accuracy)

TripType Accuracy = 73.74%


In [75]:
len(y_train)

71755

In [51]:
classes = np.array(list(set(label_enc.inverse_transform(y_labeled))))

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [52]:
dmtest = xgb.DMatrix(test.values) 
pred_proba = bst.predict(dmtest)

proba_df = pd.DataFrame(pred_proba, columns=classes)
proba_df.columns = proba_df.columns.map(lambda x: "TripType_" + str(x))
sub_df = pd.concat([test[["VisitNumber"]], proba_df], axis=1)
sub_df.to_csv("../../data/submission_0410_noProb_xgb_07.csv", index=False)

In [32]:
from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [44]:
y_labeled

array([37, 22, 18, ..., 31, 31,  5])

In [83]:
train.head()

,VisitNumber,ScanCount,Pos_Sum,Neg_Sum,CategoryCounts,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,...,Upc_Depts,Weekday_0,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Return_0.0,Return_1.0
0,5,-1,0,-1,1,0,0,0,0,0,...,1.000000,0,0,0,0,1,0,0,0,1
1,7,2,2,0,2,0,0,0,0,0,...,1.000000,0,0,0,0,1,0,0,1,0
2,8,28,30,-2,6,0,0,0,0,0,...,1.045455,0,0,0,0,1,0,0,0,1
3,9,3,3,0,2,0,0,0,0,0,...,1.000000,0,0,0,0,1,0,0,1,0
4,10,3,3,0,2,0,0,0,0,0,...,1.000000,0,0,0,0,1,0,0,1,0


In [87]:
train.isnull().sum()

VisitNumber                      0
ScanCount                        0
Pos_Sum                          0
Neg_Sum                          0
CategoryCounts                   0
1-HR PHOTO                       0
ACCESSORIES                      0
AUTOMOTIVE                       0
BAKERY                           0
BATH AND SHOWER                  0
BEAUTY                           0
BEDDING                          0
BOOKS AND MAGAZINES              0
BOYS WEAR                        0
BRAS & SHAPEWEAR                 0
CAMERAS AND SUPPLIES             0
CANDY, TOBACCO, COOKIES          0
CELEBRATION                      0
COMM BREAD                       0
CONCEPT STORES                   0
COOK AND DINE                    0
DAIRY                            0
DSD GROCERY                      0
ELECTRONICS                      0
FABRICS AND CRAFTS               0
FINANCIAL SERVICES               0
FROZEN FOODS                     0
FURNITURE                        0
GIRLS WEAR, 4-6X  AN

In [82]:
# build a classifier
clf = RandomForestClassifier(n_estimators=20)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(train, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X_train, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').